In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,5985.12,0.13,0.04,0.03,0.0,0.00,-0.00,-0.00
3,0.13,1380.41,-0.02,0.01,-0.0,0.00,-0.00,0.00
4,0.04,-0.02,306.00,0.00,0.0,-0.00,0.00,-0.00
5,0.03,0.01,0.00,57.70,0.0,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,10.5,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.0,1.87,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.0,-0.00,0.35,0.00
9,-0.00,0.00,-0.00,-0.00,0.0,0.00,0.00,0.08


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00005,0.00001,0.00000,-0.00000,-0.00002
3,0.00004,1.00000,-0.00002,0.00004,-0.00001,0.00002,-0.00005,0.00009
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00006,0.00007,-0.00006
5,0.00005,0.00004,0.00002,1.00000,0.00002,0.00002,0.00002,-0.00004
6,0.00001,-0.00001,0.00001,0.00002,1.00000,0.00002,0.00003,0.00004
7,0.00000,0.00002,-0.00006,0.00002,0.00002,1.00000,-0.00004,0.00001
8,-0.00000,-0.00005,0.00007,0.00002,0.00003,-0.00004,1.00000,0.00003
9,-0.00002,0.00009,-0.00006,-0.00004,0.00004,0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.19376321232887286

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.48303164e+06 2.33415231e+05 1.11474779e+04 3.80721345e+02
 1.18419235e+01 3.34327027e-01 9.32386037e-03 2.25486856e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999371,-0.034694,-0.007244,-0.001358,-0.000248,-0.000044,-0.000008,-0.000002
1,0.034375,0.998621,-0.039053,-0.006945,-0.001264,-0.000224,-0.000044,-0.000009
2,0.008518,0.038470,0.998426,-0.039311,-0.006862,-0.001224,-0.000229,-0.000051
3,0.001914,0.008324,0.038625,0.998067,-0.047218,-0.008058,-0.001527,-0.000333
4,0.000435,0.001888,0.008520,0.046306,0.996942,-0.061286,-0.011155,-0.002418
5,0.000102,0.000442,0.002001,0.010591,0.059401,0.993167,-0.097817,-0.020145
6,0.000028,0.000121,0.000539,0.002870,0.015779,0.090425,0.972292,-0.214976
7,0.000013,0.000054,0.000246,0.001304,0.007151,0.040245,0.212023,0.976409


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006292142532012823,
 0.001379282058734277,
 0.0015744813585429895,
 0.0019327659762049532,
 0.0030581344489504225,
 0.006832751758060973,
 0.02770822594384903,
 0.02359142440078199]